## Importa o inventário da rede Hidro da ANA

Dados importados do inventario do Hidro da ANA, segundo o documento "DESCRIÇÃO PARA DISPONIBILIZAR DADOS HIDROMETEOROLÓGICOS DOS SISTEMAS TELEMETRIA 1 E HIDRO" da ANA, disponível em https://www.ana.gov.br/telemetria1ws/Telemetria1ws.pdf

In [1]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET
from tqdm import tqdm
from datetime import datetime

A função importa e retorna a lista de todas as estações da rede Hidro, portanto, pode levar muito tempo para a execução. Deve ser utilizada apenas para atualização do data frame.

Caso queira aplicar um filtro na importação dos dados, insira o parâmetro desejado em `paramsHidro`, por exemplo: quero só os dados das estações de responsabilidade da ANA `paramsHidro = {..., 'sgResp':'ANA', ...}`, isso ajudará o processo ser mais rápido.

In [2]:
paramsHidro = {'codEstDE':'', 'codEstATE':'', 'tpEst':'', 'nmEst':'',
               'nmRio':'', 'codSubBacia':'', 'codBacia':'',
               'nmMunicipio':'', 'nmEstado':'',
               'sgResp':'', 'sgOper':'',
               'telemetrica':''}

inventarioHidro = requests.get('http://telemetriaws1.ana.gov.br/ServiceANA.asmx/HidroInventario', paramsHidro)
tree = ET.ElementTree(ET.fromstring(inventarioHidro.content))
root = tree.getroot()

Cria o data frame do inventário e exporta para um arquivo csv, assim, evita o tempo para baixar novamente os dados.

Em `columnsHidro` você pode selecionar os parâmetros de saída que deseja ser inserido no data frame.

In [3]:
 columnsHidro = ['BaciaCodigo', 'SubBaciaCodigo', 'RioCodigo',
#                'EstadoCodigo', 'MunicipioCodigo',
                 'nmEstado', 'nmMunicipio',
#                'ResponsavelCodigo', 'OperadoraCodigo',
                 'ResponsavelSigla', 'OperadoraSigla',
#                'ResponsavelUnidade', 'ResponsavelJurisdicao',
#                'OperadoraUnidade', 'OperadoraSubUnidade',
                 'TipoEstacao', 'Codigo', 'Nome',
#                'CodigoAdicional',
                 'Latitude', 'Longitude', 'Altitude', 'AreaDrenagem',
                 'TipoEstacaoEscala', 'TipoEstacaoRegistradorNivel', 'TipoEstacaoDescLiquida',
                 'TipoEstacaoSedimentos', 'TipoEstacaoQualAgua', 'TipoEstacaoPluviometro',
                 'TipoEstacaoRegistradorChuva', 'TipoEstacaoTanqueEvapo', 'TipoEstacaoClimatologica',
                 'TipoEstacaoPiezometria', 'TipoEstacaoTelemetrica',
                 'PeriodoEscalaInicio', 'PeriodoEscalaFim',
                 'PeriodoRegistradorNivelInicio', 'PeriodoRegistradorNivelFim',
                 'PeriodoDescLiquidaInicio', 'PeriodoDescLiquidaFim',
                 'PeriodoSedimentosInicio', 'PeriodoSedimentosFim',
                 'PeriodoQualAguaInicio', 'PeriodoQualAguaFim',
                 'PeriodoPluviometroInicio', 'PeriodoPluviometroFim',
                 'PeriodoRegistradorChuvaInicio', 'PeriodoRegistradorChuvaFim',
                 'PeriodoTanqueEvapoInicio', 'PeriodoTanqueEvapoFim',
                 'PeriodoClimatologicaInicio', 'PeriodoClimatologicaFim',
                 'PeriodoPiezometriaInicio', 'PeriodoPiezometriaFim',
                 'PeriodoTelemetricaInicio', 'PeriodoTelemetricaFim',
                 'TipoRedeBasica', 'TipoRedeEnergetica', 'TipoRedeNavegacao', 'TipoRedeCursoDagua',
                 'TipoRedeEstrategica', 'TipoRedeCaptacao', 'TipoRedeSedimentos', 'TipoRedeQualAgua',
                 'TipoRedeClasseVazao',
                 'UltimaAtualizacao', 'Operando',
#                'Descricao', 'NumImagens', 'DataIns', 'DataAlt'
                 ]

In [4]:
columnsDate = ['PeriodoEscalaInicio', 'PeriodoEscalaFim',
               'PeriodoRegistradorNivelInicio', 'PeriodoRegistradorNivelFim',
               'PeriodoDescLiquidaInicio', 'PeriodoDescLiquidaFim',
               'PeriodoSedimentosInicio', 'PeriodoSedimentosFim',
               'PeriodoQualAguaInicio', 'PeriodoQualAguaFim',
               'PeriodoPluviometroInicio', 'PeriodoPluviometroFim',
               'PeriodoRegistradorChuvaInicio', 'PeriodoRegistradorChuvaFim',
               'PeriodoTanqueEvapoInicio', 'PeriodoTanqueEvapoFim',
               'PeriodoClimatologicaInicio', 'PeriodoClimatologicaFim',
               'PeriodoPiezometriaInicio', 'PeriodoPiezometriaFim',
               'PeriodoTelemetricaInicio', 'PeriodoTelemetricaFim',
               'UltimaAtualizacao',
               'DataIns', 'DataAlt']
index = 1
dfHidro = pd.DataFrame()
for station in tqdm(root.iter('Table')):
    for column in columnsHidro:
        if column in columnsDate:
            if station.find(column).text is not None:
                dfHidro.at[index, column] = pd.to_datetime(datetime.strptime(station.find(column).text, '%Y-%m-%d %H:%M:%S'), errors = 'coerce')
            else:
                dfHidro.at[index, column] = None
        else:
            dfHidro.at[index, column] = getattr(station.find(column), 'text', None)
    index += 1
dfHidro.to_csv('dfHidro.csv')

38207it [14:58, 42.53it/s]


In [5]:
dfHidro.head()

,BaciaCodigo,SubBaciaCodigo,RioCodigo,nmEstado,nmMunicipio,ResponsavelSigla,OperadoraSigla,TipoEstacao,Codigo,Nome,...,TipoRedeEnergetica,TipoRedeNavegacao,TipoRedeCursoDagua,TipoRedeEstrategica,TipoRedeCaptacao,TipoRedeSedimentos,TipoRedeQualAgua,TipoRedeClasseVazao,UltimaAtualizacao,Operando
1,3,32,NaN,PARÁ,SALINÓPOLIS,INMET,INMET,2,47000,SALINÓPOLIS,...,0,0,0,0,1,0,0,0,2019-03-21,0
2,3,32,NaN,PARÁ,MARACANÃ,IDESP,IDESP,2,47001,MARACANÃ,...,0,0,9,0,3,0,0,0,2005-03-29,1
3,3,32,NaN,PARÁ,SALINÓPOLIS,ANA,CPRM,2,47002,SALINÓPOLIS,...,0,0,0,0,7,0,0,0,2018-06-18,1
4,3,32,NaN,PARÁ,CURUÇA,ANA,CPRM,2,47003,CURUÇA,...,0,0,0,0,7,0,0,0,2018-06-18,1
5,3,32,NaN,PARÁ,PRIMAVERA,ANA,CPRM,2,47004,PRIMAVERA,...,0,0,0,0,7,0,0,0,2018-02-07,1
